In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json
import jsonlines
import itertools


In [2]:
NER = ["EFFECT"]

In [3]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [4]:
df = df.reset_index().drop(columns=["index"])

In [5]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Amongst, predicted, secreted, proteins, share...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.832229375839233, -2.228951454162597, 5.12...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 8, 8, 9, ...",EFFECT
1,"[Three, R., pseudosolanacearum, strains, (, SE...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,"[[-2.702439546585083, -2.80968976020813, 6.206...","[-100, 2, 2, -100, 2, -100, -100, -100, -100, ...","[None, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4, 5, 5, ...",EFFECT
2,"[Furthermore, ,, the, isolates, with, a, diver...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.995843887329101, -3.006465196609497, 6.34...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
3,"[It, is, noteworth, mentioning, that, ICU29, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.559463262557983, -2.617972373962402, 6.08...","[-100, 2, 2, 2, -100, 2, -100, 2, 2, -100, 2, ...","[None, 0, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9, ...",EFFECT
4,"[JSC-12, is, a, filamentous, cyanobacterium, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-3.027876615524292, -2.869891405105591, 6.39...","[-100, 2, -100, -100, -100, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 0, 1, 2, 3, 4, 4, 4, 4, 4, 5, ...",EFFECT
...,...,...,...,...,...,...,...
722,"[Bacillus, anthracis, strain, Sterne, BA781, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",722,"[[-2.842869281768799, -2.509284496307373, 5.93...","[-100, 2, 2, -100, 2, 2, -100, 2, -100, -100, ...","[None, 0, 1, 1, 2, 3, 3, 4, 4, 4, 5, 6, 7, 8, ...",EFFECT
723,"[Previously, ,, we, isolated, two, strains, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",723,"[[-2.714633703231811, -2.6709907054901123, 6.5...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 11...",EFFECT
724,"[These, loci, were, then, grouped, into, diffe...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",724,"[[-3.284265995025634, -2.5439341068267822, 6.3...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
725,"[Streptococcus, agalactiae, (, GBS, ), isolate...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",725,"[[-2.86374831199646, -2.766369104385376, 5.988...","[-100, 2, 2, -100, -100, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 1, 1, 2, 3, 4, 5, 6, 6, 6, 6, ...",EFFECT


In [6]:
label_ids = df[["label_ids"]]

In [7]:
annotations = []
current_annotation = None

for j in range(len(df)):
	for i in zip(df.loc[j, "tokens"], df.loc[j, "ner_tags"]):
		if i[1] != "O":
			if current_annotation is None:
				current_annotation = { "text": i[0]}
			else:
				current_annotation["text"] += " " + i[0]
		else:
			if current_annotation is not None:
				annotations.append(current_annotation)
				current_annotation = None

if current_annotation is not None:
	annotations.append(current_annotation)


### Predict sentences again

In [8]:
ner = "EFFECT"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=0)


In [10]:
# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:
            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

Amongst predicted secreted proteins shared between pathogenic H. fraxineus and the three additional Hymenoscyphus species studied here are plant cell death inducing proteins.
[]
Three R. pseudosolanacearum strains (SEPPX05, KACC10722 and T110) were selected.
[]
Furthermore, the isolates with a diverse degree of multidrug resistance (MDR), namely NK07 (six drugs from five antimicrobial classes, 5MDR), UDRT09 (four antimicrobial classes, 4MDR), NK01 (three antimicrobial classes, 3MDR), NK02 (gentamicin resistant), and CNRT12 (resistant to only beta-lactam, Sense), were included in the subsequent genome investigation.
[{'entity_group': 'B', 'score': 0.7550202, 'word': 'multidrug', 'start': 51, 'end': 60}, {'entity_group': 'I', 'score': 0.45153898, 'word': 'resistance', 'start': 61, 'end': 71}, {'entity_group': 'B', 'score': 0.49042204, 'word': 'mdr', 'start': 73, 'end': 76}]
It is noteworth mentioning that ICU29 was isolated from a sputum sample of an 84-year-old female patient, while ICU

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[]
DQ12-45-1b was originally isolated from oil well produced liquid, where iron sources are very limited [ 38 ].
[]
In this article we describe a role for Bacillus subtilis YpsA, an uncharacterized protein of the SLOG superfamily of nucleotide and ligand-binding proteins, in cell division.
[{'entity_group': 'B', 'score': 0.7419871, 'word': 'cell', 'start': 160, 'end': 164}, {'entity_group': 'I', 'score': 0.7790391, 'word': 'division', 'start': 165, 'end': 173}]
Considering the potent anti-C. auris activity and broad spectrum of compound NMU-6, further structural optimization and SAR analysis were performed.
[{'entity_group': 'B', 'score': 0.54022723, 'word': 'anti', 'start': 23, 'end': 27}]
Due to the limited data available, the choice of potentially associated factors of FoP follows a rather explorative approach and is based on the literature on FoP in adult patients and caregivers and our clinical experience [11, 22, 24, 25].
[]
Seven of the clones are also publicly available through

In [11]:
predictions = [p for p in predictions if p]

In [12]:
predictions

[[{'entity_group': 'B',
   'score': 0.7550202,
   'word': 'multidrug',
   'start': 51,
   'end': 60},
  {'entity_group': 'I',
   'score': 0.45153898,
   'word': 'resistance',
   'start': 61,
   'end': 71},
  {'entity_group': 'B',
   'score': 0.49042204,
   'word': 'mdr',
   'start': 73,
   'end': 76}],
 [{'entity_group': 'B',
   'score': 0.86592156,
   'word': 'plant',
   'start': 123,
   'end': 128},
  {'entity_group': 'I',
   'score': 0.5887726,
   'word': 'growth - promoting',
   'start': 129,
   'end': 145}],
 [{'entity_group': 'B',
   'score': 0.7419871,
   'word': 'cell',
   'start': 160,
   'end': 164},
  {'entity_group': 'I',
   'score': 0.7790391,
   'word': 'division',
   'start': 165,
   'end': 173}],
 [{'entity_group': 'B',
   'score': 0.54022723,
   'word': 'anti',
   'start': 23,
   'end': 27}],
 [{'entity_group': 'B',
   'score': 0.5689623,
   'word': 'protection',
   'start': 270,
   'end': 280},
  {'entity_group': 'I',
   'score': 0.6097705,
   'word': 'against',
   's

In [13]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    pred = pred.replace(" - ","-").replace("- ","-").replace(" -","-")
    for annotation in annotations:
        ann = annotation["text"]
        ann = ann.lower()
        if ann in pred:
            count +=1
    if count ==0:
        print(pred)

multidrug
plant
cell
anti
protection
2
biofilm
controls
aggressive
80 % mortality in
replication
docetaxel
serum-mediated phagocytosis by human
immune dysregulation
volatile
cell
cell
cryoprotective
systemic
resistance
killing
acidic
biofilm
disease
de
high
resistance to tellurite
proliferation
growth
resistance
primary
bacterial
aggregate
biofilm
germination
lc3
xenobiotic
multi
oxidation of
soft
content
long
transcript
colony
disease
salt
bile
natural
actin
brood mortality
regular
closure
enhanced bbb
soft
prostaglandin
sufficiency
to host
invasive
antihypertensive
acid
prevention
phosphate
into
increased
phosphate
affinity
environmental
energy
metal
dna
th17
self
regulate
hydrocarbon
antioxidant
rapid
host cell contact-dependent
modulating
total antioxidant
high
rice defense
metal
outer
utilization
methylotrophy
peanut
receptor
dai index
physiological processes
nematicidal
glucose
inorganic
anti
fixing
niche
biological
facilitate
reduced
establishment
immune response
neutralizing
su

In [14]:
merged

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Amongst, predicted, secreted, proteins, share...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.832229375839233, -2.228951454162597, 5.12...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 8, 8, 9, ...",EFFECT
1,"[Three, R., pseudosolanacearum, strains, (, SE...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,"[[-2.702439546585083, -2.80968976020813, 6.206...","[-100, 2, 2, -100, 2, -100, -100, -100, -100, ...","[None, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4, 5, 5, ...",EFFECT
2,"[Furthermore, ,, the, isolates, with, a, diver...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.995843887329101, -3.006465196609497, 6.34...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
3,"[It, is, noteworth, mentioning, that, ICU29, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.559463262557983, -2.617972373962402, 6.08...","[-100, 2, 2, 2, -100, 2, -100, 2, 2, -100, 2, ...","[None, 0, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9, ...",EFFECT
4,"[JSC-12, is, a, filamentous, cyanobacterium, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-3.027876615524292, -2.869891405105591, 6.39...","[-100, 2, -100, -100, -100, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 0, 1, 2, 3, 4, 4, 4, 4, 4, 5, ...",EFFECT
...,...,...,...,...,...,...,...
722,"[Bacillus, anthracis, strain, Sterne, BA781, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",722,"[[-2.842869281768799, -2.509284496307373, 5.93...","[-100, 2, 2, -100, 2, 2, -100, 2, -100, -100, ...","[None, 0, 1, 1, 2, 3, 3, 4, 4, 4, 5, 6, 7, 8, ...",EFFECT
723,"[Previously, ,, we, isolated, two, strains, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",723,"[[-2.714633703231811, -2.6709907054901123, 6.5...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 11...",EFFECT
724,"[These, loci, were, then, grouped, into, diffe...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",724,"[[-3.284265995025634, -2.5439341068267822, 6.3...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
725,"[Streptococcus, agalactiae, (, GBS, ), isolate...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",725,"[[-2.86374831199646, -2.766369104385376, 5.988...","[-100, 2, 2, -100, -100, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 1, 1, 2, 3, 4, 5, 6, 6, 6, 6, ...",EFFECT


In [15]:
merged

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Amongst, predicted, secreted, proteins, share...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.832229375839233, -2.228951454162597, 5.12...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 8, 8, 9, ...",EFFECT
1,"[Three, R., pseudosolanacearum, strains, (, SE...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,"[[-2.702439546585083, -2.80968976020813, 6.206...","[-100, 2, 2, -100, 2, -100, -100, -100, -100, ...","[None, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4, 5, 5, ...",EFFECT
2,"[Furthermore, ,, the, isolates, with, a, diver...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-2.995843887329101, -3.006465196609497, 6.34...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
3,"[It, is, noteworth, mentioning, that, ICU29, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.559463262557983, -2.617972373962402, 6.08...","[-100, 2, 2, 2, -100, 2, -100, 2, 2, -100, 2, ...","[None, 0, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9, ...",EFFECT
4,"[JSC-12, is, a, filamentous, cyanobacterium, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-3.027876615524292, -2.869891405105591, 6.39...","[-100, 2, -100, -100, -100, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 0, 1, 2, 3, 4, 4, 4, 4, 4, 5, ...",EFFECT
...,...,...,...,...,...,...,...
722,"[Bacillus, anthracis, strain, Sterne, BA781, w...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",722,"[[-2.842869281768799, -2.509284496307373, 5.93...","[-100, 2, 2, -100, 2, 2, -100, 2, -100, -100, ...","[None, 0, 1, 1, 2, 3, 3, 4, 4, 4, 5, 6, 7, 8, ...",EFFECT
723,"[Previously, ,, we, isolated, two, strains, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",723,"[[-2.714633703231811, -2.6709907054901123, 6.5...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 10, 11...",EFFECT
724,"[These, loci, were, then, grouped, into, diffe...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",724,"[[-3.284265995025634, -2.5439341068267822, 6.3...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
725,"[Streptococcus, agalactiae, (, GBS, ), isolate...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",725,"[[-2.86374831199646, -2.766369104385376, 5.988...","[-100, 2, 2, -100, -100, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 1, 1, 1, 2, 3, 4, 5, 6, 6, 6, 6, ...",EFFECT


In [16]:
## TODO: also split the annotations by "-"
# Create an empty dataframe to store the results
results_df = pd.DataFrame(columns=['NER', 'Prediction'])

ners = ["COMPOUND", "DISEASE", "EFFECT", "ISOLATE", "MEDIUM", "METABOLITE", "ORGANISM",
        "PHENOTYPE", "PROTEIN", "SPECIES", "STRAIN"]

for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:, "ner"] = ner

    annotations = []
    current_annotation = None

    for j in range(len(merged)):
        for i in zip(merged.loc[j, "tokens"], merged.loc[j, "ner_tags"]):
            if i[1] != "O":
                if current_annotation is None:
                    current_annotation = {"text": i[0]}
                else:
                    current_annotation["text"] += " " + i[0]
            else:
                if current_annotation is not None:
                    annotations.append(current_annotation)
                    current_annotation = None

    if current_annotation is not None:
        annotations.append(current_annotation)

    path = f"../NER_output/{ner}"

    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)

    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=0)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"

    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:

                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]

    # Store the predictions in the dataframe
    for prediction in predictions:
        pred = prediction[0]["word"]
        count = 0
        pred = pred.replace(" - ", "-").replace("- ", "-").replace(" -", "-").lower()
        for annotation in annotations:
            ann = annotation["text"]
            ann = ann.lower()
            if ann in pred:
                count += 1
        if count == 0:
            results_df = pd.concat([results_df, pd.DataFrame({'NER': [ner], 'Prediction': [pred]})], ignore_index=True)

    annotations_set = [i["text"].lower().split(" ") for i in annotations]
    annotations_set = set(itertools.chain.from_iterable(annotations_set))

    predictions_set = [i[0]["word"].lower().replace(" - ", "-").replace("- ", "-").replace(" -", "-").split(" ") for i in predictions]
    predictions_set = set(itertools.chain.from_iterable(predictions_set))

    ann_not_pred = annotations_set.difference(predictions_set)
    pred_not_ann = predictions_set.difference(annotations_set)

    with open(f"ner_annotation/{ner}_ann_not_pred.txt", "w") as f:
        for i in ann_not_pred:
            f.write(i + "\n")
    with open(f"ner_annotation/{ner}_pred_not_ann.txt", "w") as f:
        for i in pred_not_ann:
            f.write(i + "\n")

/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/gomez/miniconda3/envs/torch/lib/python